<a href="https://colab.research.google.com/github/gugusx/hadoop-practice/blob/main/Hadoop_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Installing hadoop

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!java -version

In [ ]:
!sudo update-alternatives --config java

In [ ]:
!wget https://archive.apache.org/dist/hadoop/core/hadoop-3.2.3/hadoop-3.2.3.tar.gz
!tar -xzvf hadoop-3.2.3.tar.gz

In [9]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.2.3"
os.environ["PATH"] += ":/content/hadoop-3.2.3/bin:/content/hadoop-3.2.3/sbin"

In [ ]:
!hadoop version

#####  Copy paste keseluruhan text Pembukaan UUD 1945 dari website ini dari Judul sampai paragraf terakhir: https://raw.githubusercontent.com/nivdul/spark-in-practice-scala/refs/heads/master/data/wordcount.txt lalu save dengan nama wordcount.txt di local

In [ ]:
!wget -O wordcount.txt https://raw.githubusercontent.com/nivdul/spark-in-practice-scala/refs/heads/master/data/wordcount.txt

In [ ]:
%%writefile mapper.py
#!/usr/bin/env python
import sys
import re

for line in sys.stdin:
    line = line.strip()
    words = re.findall(r'\w+', line.lower())
    for word in words:
        print(f'{word}\t1')

In [ ]:
%%writefile reducer.py
#!/usr/bin/env python
import sys

current_word = None
current_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split('\t', 1)
    try:
        count = int(count)
    except ValueError:
        continue

    if current_word == word:
        current_count += count
    else:
        if current_word:
            print(f'{current_word}\t{current_count}')
        current_word = word
        current_count = count

if current_word == word:
    print(f'{current_word}\t{current_count}')

##### Running Mapper and Reducer in Hadoop Cluster

In [ ]:
!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.2.3.jar \
    -input wordcount.txt \
    -output wordcount_output \
    -mapper mapper.py \
    -reducer reducer.py

##### Save output into .txt

In [15]:
!hadoop fs -getmerge wordcount_output wordcount_result.txt